In [1]:
import os
import logging
import datetime
import json

import librosa
import pysrt

import tensorflow as tf
import numpy as np

from tensorflow.contrib import rnn

from sklearn.preprocessing import normalize as sk_normalize
from sklearn.cluster import KMeans
from scipy.ndimage.filters import gaussian_filter

import matplotlib.pyplot as plt

In [2]:
# Parameters
random_state = 123       
tdsv = False         # Text dependent or not
sr = 16000           # Sample Rate
nfft = 512           # FFT Size
window = 0.0025      # Window Lenght 25ms
hop = 0.01           # Hop Size 10ms
tdsv_frame = 160     # Frame number of utterance of tdsv
tisv_frame = 160     # Max frame number of utterances of tdsv
tisv_frame_min = 50  # min frame length to confirm with 'd-vector v-3 model'

# Model Parameters
hidden = 768        # Hidden State dimension of LSTM
proj = 256          # Hidden State dimension of LSTM
num_layer = 3       # Number of LSTM layers
restore = False     # Restore model or not
model_num = 3       # Number of ckpt file to load 

# Main Parameters
number_of_speakers = 2   # Number of speakers
srt_path = 'output-files/str_file.en.srt' # Enter the absolute path to srt file
log_path = 'output-files/main.logs' # 
output_dir = 'output-files/output.json' # Path to save the output json file
audio_file = '/home/bruno/datasets/mixed-musan/wav16/'
model_path = 'models/model.ckpt-46'
output_path = 'output-files/mixed-musan/'

if not os.path.isdir(output_path):
    os.makedirs(output_path)

vad_threshold = 25

In [3]:
def normalize(x):
    """ normalize the last dimension vector of the input matrix
    :return: normalized input
    """
    return x/tf.sqrt(tf.reduce_sum(x**2, axis=-1, keep_dims=True)+1e-6)

In [4]:
all_speakers = []
for path, dirs, files in sorted(os.walk(audio_file)):
    for file in files:
        if file.endswith('.wav'):
            all_speakers.append(os.path.join(path,file))
all_speakers = sorted(all_speakers) 

In [ ]:
for audio_index, audio_path in enumerate(all_speakers): 
    tf.reset_default_graph()
    batch_size = 2 # Fixing to 2 since we take 2 for each interval #utter_batch.shape[1]

    verif = tf.placeholder(shape=[None, batch_size, 40], dtype=tf.float32)  # verification batch (time x batch x n_mel)
    batch = tf.concat([verif,], axis=1)
    # embedding lstm (3-layer default)


    with tf.variable_scope("lstm"):
        lstm_cells = [tf.contrib.rnn.LSTMCell(num_units=hidden, num_proj=proj) for i in range(num_layer)]
        lstm = tf.contrib.rnn.MultiRNNCell(lstm_cells)    # make lstm op and variables
        outputs, _ = tf.nn.dynamic_rnn(cell=lstm, inputs=batch, dtype=tf.float32, time_major=True)   # for TI-VS must use dynamic rnn
        embedded = outputs[-1]                            # the last ouput is the embedded d-vector
        embedded = normalize(embedded)                    # normalize

    config_tensorflow = tf.ConfigProto(
            device_count = {'GPU': 0}
        )
    saver = tf.train.Saver(var_list=tf.global_variables())
    
    fileName = audio_path.split('/')[-1].split('.')[0]   
    # Extract embeddings
    # Each embedding saved file will have (2, 256)
    with tf.Session(config=config_tensorflow) as sess:
        tf.global_variables_initializer().run()
        saver.restore(sess, model_path)
        logging.info("loading audio")
        
        utter, sr = librosa.core.load(audio_path, sr=sr)        # load audio
        utter_min_len = (tisv_frame_min * hop + window) * sr    # lower bound of utterance length
        # Get the duration
        duration = librosa.get_duration(utter, sr)
        # Duration of each window
        duration_per_frame = (duration / utter.shape[0])
        logging.info(f'Duration: {duration}\nDuration per frame: {duration_per_frame}s\nMin length of utterance: {utter_min_len * duration_per_frame}s')
        tisv_frame_duration_s = utter_min_len * duration_per_frame
        intervals = librosa.effects.split(utter, top_db=vad_threshold)         # voice activity detection

        all_data = []
        logging.info('Converting intervals to embeddings')
        selected_intervals_idx = []
        for idx, current_interval in enumerate(intervals):
            if (current_interval[1]-current_interval[0]) > utter_min_len:
                # Save these selected intervals, as shorter ones are ignored
                selected_intervals_idx.append(idx)
                utterances_spec = []
                utter_part = utter[current_interval[0]:current_interval[1]]         # save first and last 160 frames of spectrogram.
                S = librosa.core.stft(y=utter_part, n_fft=nfft,
                                        win_length=int(window * sr), hop_length=int(hop * sr))
                S = np.abs(S) ** 2
                mel_basis = librosa.filters.mel(sr=sr, n_fft=nfft, n_mels=40)
                S = np.log10(np.dot(mel_basis, S) + 1e-6)           # log mel spectrogram of utterances
                utterances_spec.append(S[:, :tisv_frame])
                utterances_spec.append(S[:, -tisv_frame:])
                utterances_spec = np.array(utterances_spec)
                utter_batch = np.transpose(utterances_spec, axes=(2,0,1))     # transpose [frames, batch, n_mels]

                data = sess.run(embedded, feed_dict={verif:utter_batch})
                all_data.extend(data)
    data = np.array(all_data)

    ######### Spectral Clustering #########

    # Cossine Similarity

    similarity = np.dot(data, data.T)  # Cossine similarity
    square_mag = np.diag(similarity)   # Squared magnitude of preference vectors (number of occurrences) (diagonals are ai*ai)
    inv_square_mag = 1 / square_mag    # Inverse squared magnitude
    inv_square_mag[np.isinf(inv_square_mag)] = 0 # if it doesn't occur, set it's inverse magnitude to zero (instead of inf)
    inv_mag = np.sqrt(inv_square_mag)  # Inverse of the magnitude
    cosine = similarity * inv_mag      # Cosine similarity (elementwise multiply by inverse magnitudes)
    A =  cosine.T * inv_mag            # Final step cossine sim
    np.fill_diagonal(A, -1000)         # Fill the diagonals with very large negative value
    np.fill_diagonal(A, A.max(axis=1)) # Fill the diagonals with the max of each row
    A = (1-A)/2                        # Final step in cossine sim

    # Gaussian blur
    sigma = 0.5  # we will select sigma as 0.5
    A_gau = gaussian_filter(A, sigma)

    # para vizialização
    A_gau_sigma1 = gaussian_filter(A, 1)
    A_gau_sigma2 = gaussian_filter(A, 2)

    # Symmetrization

    threshold_multiplier = 0.01  # Thresholding using multiplier = 0.01
    A_thresh = A_gau * threshold_multiplier# Symmetrization
    A_sym = np.maximum(A_thresh, A_thresh.T)

    # Diffusion
    A_diffusion = A_sym * A_sym.T # Row-wise matrix Normalization
    Row_max = A_diffusion.max(axis=1).reshape(1, A_diffusion.shape[0]) 

    # Row-wise Max Normalization: 
    A_norm = A_diffusion / Row_max.T

    # Eigen decomposition
    eigval, eigvec = np.linalg.eig(A_norm)
    # Since eigen values cannot be negative for Positive semi definite matrix, the numpy returns negative values, converting it to positive
    eigval = np.abs(eigval)
    sorted_eigval_idx = np.argsort(eigval)[::-1]
    sorted_eigval = np.sort(eigval)[::-1]
    # For division according to the equation
    eigval_shifted = np.roll(sorted_eigval, -1)
    # Thresholding eigen values because we don't need very low eigan values due to errors
    eigval_thresh = 0.1
    sorted_eigval = sorted_eigval[sorted_eigval > eigval_thresh]
    eigval_shifted = eigval_shifted[:sorted_eigval.shape[0]]
    # Don't take the first value for calculations, if first value is large, following equation will return k=1, and we want more than one clusters
    # Get the argmax of the division, since its 0 indexed, add 1
    k = np.argmax(sorted_eigval[1:]/eigval_shifted[1:]) + 2
    logging.debug(f'Number of Eigen vectors to pick: {k}')
    # Get the indexes of eigen vectors
    idexes = sorted_eigval_idx[:k]
    A_eigvec = eigvec[:, idexes]
    A_eigvec = A_eigvec.astype('float32')

    print(f'Number of cluster: {k}')
    print(A_eigvec.shape)

    # K-means

    A_eigvec_norm = sk_normalize(A_eigvec) # l2 normalized
    kmeans = KMeans(n_clusters=number_of_speakers, init='k-means++', random_state=random_state)
    kmeans.fit(A_eigvec)
    labels = kmeans.labels_

    #output_srt_json = os.path.join(output_dir, os.path.basename(srt_path) + '.json')
    #output_wav_json = os.path.join(output_dir, os.path.basename(srt_path) + '.wav.json')
    output_srt_json = os.path.join(output_dir, fileName + '_' + os.path.basename(srt_path) + '.json')
    output_wav_json = os.path.join(output_path, fileName + '.wav.json')  

    OL_INDICATOR = 'OL'
    SIL_INDICATOR = -1
    json_data = []
    for idx, i in enumerate(selected_intervals_idx):
        start = str(datetime.timedelta(seconds = intervals[i][0] * duration_per_frame))
        end = str(datetime.timedelta(seconds = intervals[i][1] * duration_per_frame))
        speaker = labels[idx*2]
        if labels[idx*2] != labels[(idx*2)+1]:
            speaker = 'OL' # possible overlap
        json_data.append({
            'start': start,
            'end': end,
            'speaker': str(speaker)
        })

    # Save the output to json
    print('Saving file:', fileName)
    with open(output_wav_json, 'w') as f:
        json.dump(json_data, f, indent=4)


    complete_json = {}
    json_data = []
    subs = pysrt.open(output_wav_json, encoding="utf-8")
    convert_to_ms = lambda st: (st.hours * 60 * 60 * 1000) + \
                                (st.minutes * 60 * 1000) +\
                                (st.seconds * 1000) +\
                                (st.milliseconds)
    for sub in subs:
        start_in_ms = convert_to_ms(sub.start)
        end_in_ms = convert_to_ms(sub.end)
        speakers = []
        for idx, i in enumerate(selected_intervals_idx):
            start = intervals[i][0] * duration_per_frame * 1000
            end = intervals[i][1] * duration_per_frame * 1000
            if start_in_ms <= start <= end_in_ms:
                speaker = int(labels[idx*2])
                if labels[idx*2] != labels[(idx*2)+1]:
                    speaker = OL_INDICATOR # possible overlap
                speakers.append(speaker)
        json_data.append({
            "index": sub.index,
            "start": sub.start.to_time().strftime("%H:%M:%S,%f")[:-3],
            "end": sub.end.to_time().strftime("%H:%M:%S,%f")[:-3],
            'speakers': np.unique(speakers).tolist(),
            'speakers_distribution': speakers,
            'text': sub.text
        })
    metadata = {
        "overlap_indicator": OL_INDICATOR,
        "duration": duration,
        "class_names": np.unique(labels).tolist(),
        "num_of_speakers": len(set(labels)),
        "silence_indicator": SIL_INDICATOR
    }
    complete_json["metadata"] = metadata
    complete_json["srt"] = json_data
    # Save the output to json
    with open(output_srt_json, 'w') as f:
        json.dump(complete_json, f, indent=4)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Restoring parameters from models/model.ckpt-46
Number of cluster: 2
(772, 2)
Saving file: two_speakers_0
INFO:tensorflow:Restoring parameters from models/model.ckpt-46


INFO:tensorflow:Restoring parameters from models/model.ckpt-46
/home/bruno/miniconda3/envs/diarization-test/lib/python3.6/site-packages/ipykernel_launcher.py:118: ComplexWarning: Casting complex values to real discards the imaginary part


Number of cluster: 2
(918, 2)
Saving file: two_speakers_1
INFO:tensorflow:Restoring parameters from models/model.ckpt-46


INFO:tensorflow:Restoring parameters from models/model.ckpt-46


Number of cluster: 2
(822, 2)
Saving file: two_speakers_10
INFO:tensorflow:Restoring parameters from models/model.ckpt-46


...

INFO:tensorflow:Restoring parameters from models/model.ckpt-46


Number of cluster: 3
(682, 3)
Saving file: two_speakers_72
INFO:tensorflow:Restoring parameters from models/model.ckpt-46


INFO:tensorflow:Restoring parameters from models/model.ckpt-46


Number of cluster: 2
(376, 2)
Saving file: two_speakers_73
INFO:tensorflow:Restoring parameters from models/model.ckpt-46


INFO:tensorflow:Restoring parameters from models/model.ckpt-46
